# 0-dataset-module1-vectorstoreretriever

- parent 관련된 모듈
    1. https://asidefine.tistory.com/298
    2. https://wikidocs.net/234164
- multi-vector retriever
    1. https://wikidocs.net/234281

### 1. datapath & read

json을 일단 읽어준다.

In [1]:
import os
from dotenv import load_dotenv
import openai
import json
import logging #logging for queries
# from langchain_community.document_loaders import TextLoader
# from langchain.document_loaders import JSONLoader

# load api keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

PATH_TEST = "../egogoal/v2/annotations/goalstep_test_unannotated.json"
PATH_TRAIN = "../egogoal/v2/annotations/goalstep_train.json"
PATH_VAL = "../egogoal/v2/annotations/goalstep_val.json"
PATH_TRAINVAL = "../egogoal/v2/annotations/goalstep_trainval.json"

data_test = json.load(open(PATH_TEST))
data_train = json.load(open(PATH_TRAIN))
data_val = json.load(open(PATH_VAL))

train_video_uids = set(x.get("video_id", x.get("video_uid", None)) for x in data_test["annotations"])
val_video_uids = set(x.get("video_id", x.get("video_uid", None)) for x in data_train["annotations"])
test_video_uids = set(x.get("video_id", x.get("video_uid", None)) for x in data_val["annotations"])


FileNotFoundError: [Errno 2] No such file or directory: './egogoal/v2/goalstep_test_unannotated.json'

### 2. make hierarchical 

어떠한 chunk에 대한 유사도가 감지되었다면, 그 chunk를 가지고 있는 원본 document를 모두 참고하는 방법이다.
- 각 비디오는 separate document로 documents에 들어가야 한다.
- 각 비디오가 많이 길수가 있으니까 역시 chunk를 만들어서 줘야 한다.
- 다만, 맨 처음의 spatial context는 무조건 줘야 한다.

In [ ]:
# Loading and Splitting
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever


# split dataset into lists
documents = []
embeddings = OpenAIEmbeddings()

# text splitter
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
vectorstore = Chroma(
    collection_name="full_docs", embedding = embeddings
)

# storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

### 3. Retrieve from documents list

In [ ]:
QUERY = ""

# add documents to retriever and see how many chunk the full document has
retriever.add_documents(documents)
len(list(store.yield_keys()))

# check if search is possible for smaller chunks
sub_docs = vectorstore.similarity_search(QUERY)
print(sub_docs[0].page_content)
len(sub_docs[0].page_content)

# two ways of retrieving documents
retrieved_docs = retriever.invoke(QUERY) 
retrieved_docs = retriever.get_relevant_documents(QUERY)

# print the searched document
print(retrieved_docs[0].page_content)
len(retrieved_docs[0].page_content)

### 4. Store VectorStore